In [1]:
import pandas as pd
import numpy as np
import re
from googleapiclient.discovery import build

import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.metrics import jaccard_distance
#nltk.download('wordnet')
#nltk.download('stopwords')

   + Open my trending channel database

In [2]:
file_path = 'C:/Users/Andrea/Documents/DATA/YT_Channel_Recommender/Database/df_trending.csv'
df_trending= pd.read_csv(file_path)
#df_trending
df_trending.shape
df_trending.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706 entries, 0 to 705
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   channel_id     706 non-null    object
 1   channel_title  706 non-null    object
 2   category_id    706 non-null    int64 
 3   title          706 non-null    object
 4   tags           706 non-null    object
 5   description    643 non-null    object
dtypes: int64(1), object(5)
memory usage: 33.2+ KB


In [3]:
df_trending['tags'].isna().sum()

0

In [4]:
df_trending['category_id'] = df_trending['category_id'].astype('object')
#df_trending.info()
#df_trending.head()

   + Active my credentials for the YouTube v3 api

In [5]:
api_key = open("api_key_1.txt","r")
api_key = api_key.read()
youtube = build('youtube', 'v3', developerKey=api_key)

   + Function to get the Video ID only from the User input using youtube.search.list()

In [6]:
def get_video_id(user_input):
    request = youtube.search().list(
        part="snippet",
        maxResults=1,
        q=user_input
    )
    response = request.execute()

    items = response['items']
    video_ids = []  
    for item in items:
        video_id = item['id']['videoId']
        video_ids.append(video_id)  
    
    if len(video_ids) > 0:
        return video_ids
    else:
        print("Sorry, I can't find your topic in my recommendations. Let's try another topic :).")
        return None

user_input = input("Type a topic for your channel: ")
video_ids = get_video_id(user_input)

#print(video_ids)


Type a topic for your channel: Football boots


   + Function to get the tags and category ID from youtube.video.list()

In [7]:
def get_tags_channel(video_ids):
    request = youtube.videos().list(
        part="snippet",
        id=video_ids
    )
    response = request.execute()

    database = []

    items = response['items']
    for item in items:
        video_data = [
            item['snippet'].get('channelId'),
            item['snippet'].get('channelTitle'),
            item['snippet'].get('categoryId'),
            item['snippet'].get('title'),
            item['snippet'].get('tags'),
            item['snippet'].get('description')
        ]

        channel_url = item['snippet'].get('channelId')
        request_1 = youtube.channels().list(
            part="snippet",
            id=channel_url
        )
        response_1 = request_1.execute()

        channel_data = response_1['items'][0]['snippet']
        video_data.append(channel_data.get('customUrl'))

        database.append(video_data)

    return database

database = get_tags_channel(video_ids)

df_channel = pd.DataFrame(database, columns=['channel_id', 'channel_title', 'category_id', 'title', 'tags', 'description', 'custom_url'])

def clean_up(tag_list):
    cleaned_tags = []
    for tag in tag_list:
        tag = str(tag).lower()
        tag = re.sub("http:\S+", " ", tag)
        tag = re.findall("[a-z]+", tag)
        cleaned_tags.extend(tag)
    return ','.join(cleaned_tags)

df_channel['tags'] = df_channel['tags'].apply(clean_up)

def tokenize(tags):
    return word_tokenize(str(tags))

df_channel['tags'] = df_channel['tags'].apply(tokenize)

def lemmatize_input(tags):
    lem = WordNetLemmatizer()
    
    return [lem.lemmatize(word) for word in tags]  

df_channel['tags'] = df_channel['tags'].apply(lemmatize_input)

english_stopwords = stopwords.words('english')
spanish_stopwords = stopwords.words('spanish')

all_stopwords = english_stopwords + spanish_stopwords

def remove_sw_input(tags):
    return [word for word in tags if not word in all_stopwords]  

df_channel['tags'] = df_channel['tags'].apply(remove_sw_input)

def clean_tags(tags):
    tags = [tag for tag in tags if tag != ',']
    tags = list(set(tags))
    return tags

df_channel['tags'] = df_channel['tags'].apply(clean_tags)


df_channel

,channel_id,channel_title,category_id,title,tags,description,custom_url
0,UC5SQGzkWyQSW_fe-URgq7xw,Unisport,17,5 mistakes to AVOID when buying new boots,"[soccer, unisport, boot, skill, tutorial, clea...",AVOID these 5 mistakes when you're buying new ...,@unisport


In [8]:
#Check the tags values
df_channel['tags'].values

array([list(['soccer', 'unisport', 'boot', 'skill', 'tutorial', 'cleat', 'football', 'learn', 'review'])],
      dtype=object)

In [9]:
#df_channel.isna().sum()

   * First step is search for coincidences in my trending dataset

In [10]:
categories_df_channel = set(df_channel['category_id'])
categories_df_trending = set(df_trending['category_id'])
print(categories_df_channel - categories_df_trending) 
print(categories_df_trending - categories_df_channel) 

{'17'}
{1, 2, 10, 17, 19, 20, 22, 23, 24, 25, 26, 27, 28}


   + Secound step: Filter by category id 

In [11]:
category = int(df_channel['category_id'].iloc[0])
filtered_df_trending = df_trending[df_trending['category_id'] == category].copy()
filtered_df_trending.head()
#print(filtered_df_trending.shape)

,channel_id,channel_title,category_id,title,tags,description
3,UCuN9hYw2RpoAW8rZ3VK3isA,NASCAR,17,Chicago Street Race: Grant Park 220 Extended H...,"['nascar', 'nascar cup series', 'chicago stree...",relive the inaugural grant park 220 that saw a...
11,UCKwGZZMrhNYKzucCtTPY2Nw,PGA TOUR,17,Highlights | Round 4 & Playoff | Rocket Mortga...,"['2023', '2023 rocket mortgage classic', 'detr...",check out the best shots of the day from the f...
15,UCRijo3ddMTht_IHyNSNXpNQ,Dude Perfect,17,"$50,000 Crystal Treasure Hunt","['dude perfect', 'dude perfect stereotypes', '...",crystal hunting round 2 \n thanks for subscri...
18,UCVSSpcmZD2PwPBqb8yKQKBA,NBA on ESPN,17,Woj's NBA free agency update on Damian Lillard...,"['espn', 'nba', 'nba espn', 'espn nba', 'nba o...",wojs nba free agency update on damian lillard ...
21,UC5igJFdBQVqg7hXFI7075OQ,Golf Channel,17,PGA Tour Highlights: 2023 Rocket Mortgage Clas...,"['golf', 'sports', 'golf channel', 'rocket mor...",check out the best shots and moments from the ...


In [13]:
def filter_trending_df(df_trending, category):
    filtered_df_trending = df_trending[df_trending['category_id'] == category].copy()
    #print(filtered_df_trending.head())
    #print(filtered_df_trending.shape)
    
    if filtered_df_trending.shape[0] < 10:
        df_channel_filtered = df_channel[['channel_title', 'custom_url']].copy()
        #print(df_channel_filtered)
        print("Sorry, we found just one channel. Please try again.")
    
    return filtered_df_trending

category = int(df_channel['category_id'].iloc[0])
filtered_df_trending = filter_trending_df(df_trending, category)
filtered_df_trending

,channel_id,channel_title,category_id,title,tags,description
3,UCuN9hYw2RpoAW8rZ3VK3isA,NASCAR,17,Chicago Street Race: Grant Park 220 Extended H...,"['nascar', 'nascar cup series', 'chicago stree...",relive the inaugural grant park 220 that saw a...
11,UCKwGZZMrhNYKzucCtTPY2Nw,PGA TOUR,17,Highlights | Round 4 & Playoff | Rocket Mortga...,"['2023', '2023 rocket mortgage classic', 'detr...",check out the best shots of the day from the f...
15,UCRijo3ddMTht_IHyNSNXpNQ,Dude Perfect,17,"$50,000 Crystal Treasure Hunt","['dude perfect', 'dude perfect stereotypes', '...",crystal hunting round 2 \n thanks for subscri...
18,UCVSSpcmZD2PwPBqb8yKQKBA,NBA on ESPN,17,Woj's NBA free agency update on Damian Lillard...,"['espn', 'nba', 'nba espn', 'espn nba', 'nba o...",wojs nba free agency update on damian lillard ...
21,UC5igJFdBQVqg7hXFI7075OQ,Golf Channel,17,PGA Tour Highlights: 2023 Rocket Mortgage Clas...,"['golf', 'sports', 'golf channel', 'rocket mor...",check out the best shots and moments from the ...
31,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,17,The Usos vs. Roman Reigns & Solo Sikoa - The B...,"['jey uso', 'jimmy uso', 'paul heyman', 'roman...",the bloodline civil war erupts as the usos bat...
50,UCwD9E_QNwFwrQC7OnPnrg2Q,Guardian Sport,17,Ashes captains have their say on Bairstow stum...,"['alex carey', 'alex carey stumping', 'ashes',...",speaking after the second ashes test at lords ...
58,UCSpycUnuU0IVF7gGIhGojhg,Tour de France,17,Highlights - Stage 2 - Tour de France 2023,"['#tdf', 'alaphilippe', 'bernal', 'bilbao', 'c...",discover the stage 2 highlights\r\n more infor...
64,UC_JQGBtA7P0RwkRxd7xpJcA,Sky Sports Boxing,17,HIGHLIGHTS! Franchón Crews-Dezurn vs Savannah ...,"['sky sports boxing', 'sky sports boxing strea...",subscribe \nhighlights as savannah marshall be...
88,UCvgfXK4nTYKudb0rFR6noLA,UFC - Ultimate Fighting Championship,17,Things are beginning to heat up between these ...,"['ufc', 'mma', 'ultimate fighting championship...",subscribe to get all the latest ufc content \n...


In [ ]:
filtered_df_trending['tags'].dtype

   + NLP cleaning

In [14]:
def tokenizer_and_remove_punctuation(row):
    tokens = word_tokenize(row['tags'])
    return [word.lower() for word in tokens if word.isalnum()]

filtered_df_trending['tags']= filtered_df_trending.apply(tokenizer_and_remove_punctuation,axis=1)
filtered_df_trending.head()

,channel_id,channel_title,category_id,title,tags,description
3,UCuN9hYw2RpoAW8rZ3VK3isA,NASCAR,17,Chicago Street Race: Grant Park 220 Extended H...,"[cup, series, street, race, street, race, gran...",relive the inaugural grant park 220 that saw a...
11,UCKwGZZMrhNYKzucCtTPY2Nw,PGA TOUR,17,Highlights | Round 4 & Playoff | Rocket Mortga...,"[rocket, mortgage, classic, golf, club, tour, ...",check out the best shots of the day from the f...
15,UCRijo3ddMTht_IHyNSNXpNQ,Dude Perfect,17,"$50,000 Crystal Treasure Hunt","[perfect, perfect, stereotypes, perfect, water...",crystal hunting round 2 \n thanks for subscri...
18,UCVSSpcmZD2PwPBqb8yKQKBA,NBA on ESPN,17,Woj's NBA free agency update on Damian Lillard...,"[espn, nba, on, espn, damian, lillard, lillard...",wojs nba free agency update on damian lillard ...
21,UC5igJFdBQVqg7hXFI7075OQ,Golf Channel,17,PGA Tour Highlights: 2023 Rocket Mortgage Clas...,"[channel, mortgage, classic, highlights, mortg...",check out the best shots and moments from the ...


In [15]:
filtered_df_trending.isna().sum()

channel_id       0
channel_title    0
category_id      0
title            0
tags             0
description      1
dtype: int64

In [16]:
def lemmatize(row):
    lem = WordNetLemmatizer()
    
    return [lem.lemmatize(word) for word in row['tags']]

filtered_df_trending['tags'] = filtered_df_trending.apply(lemmatize,axis=1)
filtered_df_trending.head()
filtered_df_trending.shape

(53, 6)

In [17]:
%%time
english_stopwords = stopwords.words('english')
spanish_stopwords = stopwords.words('spanish')

all_stopwords = english_stopwords + spanish_stopwords

def remove_sw(row):
    return [word for word in row['tags'] if not word in all_stopwords]

filtered_df_trending['tags'] = filtered_df_trending.apply(remove_sw, axis=1)

def clean_tags(tags):
    tags = [tag for tag in tags if tag != ',']
    tags = list(set(tags))
    
    return tags

filtered_df_trending['tags'] = filtered_df_trending['tags'].apply(clean_tags)

filtered_df_trending.head()


CPU times: total: 31.2 ms
Wall time: 14 ms


,channel_id,channel_title,category_id,title,tags,description
3,UCuN9hYw2RpoAW8rZ3VK3isA,NASCAR,17,Chicago Street Race: Grant Park 220 Extended H...,"[cup, nbc, gisbergen, elliott, ringer, win, st...",relive the inaugural grant park 220 that saw a...
11,UCKwGZZMrhNYKzucCtTPY2Nw,PGA TOUR,17,Highlights | Round 4 & Playoff | Rocket Mortga...,"[shot, tour, day, best, final, classic, golf, ...",check out the best shots of the day from the f...
15,UCRijo3ddMTht_IHyNSNXpNQ,Dude Perfect,17,"$50,000 Crystal Treasure Hunt","[bottle, world, flip, record, shot, perfect, w...",crystal hunting round 2 \n thanks for subscri...
18,UCVSSpcmZD2PwPBqb8yKQKBA,NBA on ESPN,17,Woj's NBA free agency update on Damian Lillard...,"[rumor, wojnarowski, nba, trade, sabonis, lill...",wojs nba free agency update on damian lillard ...
21,UC5igJFdBQVqg7hXFI7075OQ,Golf Channel,17,PGA Tour Highlights: 2023 Rocket Mortgage Clas...,"[shot, tour, fowler, best, moment, classic, pg...",check out the best shots and moments from the ...


In [ ]:
#filtered_df_trending['tags']

   + Find the similarity

In [18]:
#Test two different operations
def calculate_similarity(tags1, tags2):
    set_tags1 = set(tags1)
    set_tags2 = set(tags2)
    score = len(set_tags1.intersection(set_tags2)) / len(set_tags1.union(set_tags2))
    return score

def calculate_score(row):
    tags_channel = df_channel['tags'][0]
    tags_trending = row['tags']
    score = calculate_similarity(tags_channel, tags_trending)
    return score

filtered_df_trending['score'] = filtered_df_trending.apply(calculate_score, axis=1)
filtered_df_trending['score'].values


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.09090909, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.04      , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.04347826, 0.05555556, 0.05263158, 0.05263158, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.11764706, 0.        , 0.        , 0.        ,
       0.04166667, 0.        , 0.16666667])

In [ ]:
from nltk.metrics import jaccard_distance

def calculate_similarity(tags1, tags2):
    set_tags1 = set(tags1)
    set_tags2 = set(tags2)
    score = 1 - jaccard_distance(set_tags1, set_tags2)
    return score

def calculate_score(row):
    tags_channel = df_channel['tags'][0]
    tags_trending = row['tags']
    score = sum(calculate_similarity(tags_channel, tags_trending) for word_trending in tags_trending)
    return score

filtered_df_trending['score'] = filtered_df_trending.apply(calculate_score, axis=1)
filtered_df_trending['score'].values

In [19]:
filtered_df_trending.sort_values('score', ascending=False)

,channel_id,channel_title,category_id,title,tags,description,score
558,UClgtzzH6GlVZc4MreD3BJdQ,Futbol y Curvas,17,Fútbol Comedia I Trata De No Reírte 🤭,"[skill, sport, moment, football, highlight]",🚨 Si te ríes pierdes nivel DIOS 🙊\n\n👉 English...,0.166667
508,UCET00YnetHT7tOpu12v8jxg,CBS Sports Golazo,17,USA vs. Mexico: Extended Highlights | CONCACAF...,"[soccer, richards, studio, futbol, league, gol...",Mexico take on the USA and Christian Pulisic i...,0.117647
101,UCLzocFrP98gfI5W2LD7Ux-w,247Sports,17,Why you ALWAYS play to the whistle 🏈 #shorts,"[football, ute, duck]",collegefootball football utahfootball utahutes...,0.090909
483,UCRQMZQ0R561looJMoxPmRoQ,Spor Delisi HD,17,Impossible Moments in Sports,"[imkansız, sport, moment, nba, golf, football,...",Impossible Moments in Sports video with you. Y...,0.055556
488,UCb4_z-mud9YU0b0ine0vs1A,SportsPro,17,1 IN A TRILLION MOMENTS IN SPORTS,"[vine, billion, sport, fails, moment, football...",1 IN A TRILLION MOMENTS IN SPORTS\n\n🔔 Support...,0.052632
490,UCVuuI7QB8ViizzVAH3L_H7Q,WTD Productions,17,Greatest Sports Moments,"[10, production, time, u, sport, amazing, mome...",UPDATE: I AM IN THE PROCESS OF MAKING REVISION...,0.052632
482,UC2t9imwIR7xlDfma3CmRKJQ,T Sports,17,⚽Live | Bangladesh vs Kuwait | SAFF Championsh...,"[cricket, championship, sport, live, banglades...",⚽Live | Bangladesh vs Kuwait| SAFF Championshi...,0.043478
512,UCNAf1k0yIjyGu3k9BwAg3lg,Sky Sports Premier League,17,"TRANSFER UPDATE | Latest on Declan Rice, Mount...","[arsenal, city, day, league, sport, live, tran...",SUBSCRIBE ► https://bit.ly/SubscribeSkySportsP...,0.041667
166,UCJ2F8qpzHRk3RdJZ4YXLbxQ,Selección Española de Fútbol (SeFutbol),17,RESUMEN | España 2-1 Suiza | Cuartos de final ...,"[team, u21euro, stadium, entrenamiento, futbol...",suscríbete al canal de la selección española d...,0.040000
493,UCV5HXUHFR-yuSb7qDJ5_80g,Wide World of Sports,17,| Wide World of Sports,"[world, nine, maroon, league, sport, origin, b...",FULL REPLAYS AND MINIS: https://nine.social/eI...,0.000000


In [20]:
#Check the tags in the high score row
tags_values = filtered_df_trending.loc[508, 'tags']
tags_values

['soccer',
 'richards',
 'studio',
 'futbol',
 'league',
 'golazo',
 'football',
 'highlight',
 'henry',
 'carragher']

In [21]:
# Order by descending
top_3_scores = filtered_df_trending.sort_values('score', ascending=False).head(3)
top_3_scores

,channel_id,channel_title,category_id,title,tags,description,score
558,UClgtzzH6GlVZc4MreD3BJdQ,Futbol y Curvas,17,Fútbol Comedia I Trata De No Reírte 🤭,"[skill, sport, moment, football, highlight]",🚨 Si te ríes pierdes nivel DIOS 🙊\n\n👉 English...,0.166667
508,UCET00YnetHT7tOpu12v8jxg,CBS Sports Golazo,17,USA vs. Mexico: Extended Highlights | CONCACAF...,"[soccer, richards, studio, futbol, league, gol...",Mexico take on the USA and Christian Pulisic i...,0.117647
101,UCLzocFrP98gfI5W2LD7Ux-w,247Sports,17,Why you ALWAYS play to the whistle 🏈 #shorts,"[football, ute, duck]",collegefootball football utahfootball utahutes...,0.090909


   + Connect API to get the Custom URL

In [22]:
def get_custom_url():
    channel_url = []
    for channel_id in top_3_scores['channel_id']:
        request = youtube.channels().list(
            part="snippet",
            id=channel_id
        )
        response = request.execute()
        items = response['items']
        for item in items:
            custom_url = item['snippet'].get('customUrl')
            channel_url.append(custom_url)
    return channel_url

channel_url = get_custom_url()
#channel_url

top_3_scores['channel_url'] = channel_url
top_3_scores = top_3_scores.drop(columns=['category_id', 'title', 'tags', 'description','score'])
top_3_scores

,channel_id,channel_title,channel_url
558,UClgtzzH6GlVZc4MreD3BJdQ,Futbol y Curvas,@futbolycurvas
508,UCET00YnetHT7tOpu12v8jxg,CBS Sports Golazo,@cbssportsgolazo
101,UCLzocFrP98gfI5W2LD7Ux-w,247Sports,@247sports
